<a href="https://colab.research.google.com/github/kojihashimoto-kobe/colab_sample/blob/main/fasterwhisper%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install git+https://github.com/kojihashimoto-kobe/faster-whisper.git
!pip install transformers==4.27.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 13.5 kB 135.6 kB/s 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for faster-whisper: filename=faster_whisper-0.1.0-py3-none-any.whl size=13970 sha256=cacf16eb58766fd85539e2879cb1c953b61e4f15868a9e9466dff1b1b2e0ab95
  Stored in directory: /tmp/pip-ephem-wheel-cache-b09cq6nn/wheels/c8/2c/e9/8959df87a488ee673470cf5721ec7e37690fd032d8065e9610
Successfully built faster-whisper
  Attempting uninstall: faster-whisper
    Found existing installation: faster-whisper 0.6.0
    Uninstalling faster-whisper-0.6.0:
      Successfully uninstalled faster-whisper-0.6.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install gradio==3.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import gradio as gr
# whisper large-v2をhugging faceから取得して、ctranslate2でC++に変換&パラメータなどの量子化する
from faster_whisper import WhisperModel
!ct2-transformers-converter --model openai/whisper-large-v2 --output_dir whisper-large-v2-ct2 \
    --copy_files tokenizer.json --quantization float16

model_path = "whisper-large-v2-ct2/"

Traceback (most recent call last):
  File "/usr/local/bin/ct2-transformers-converter", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/ctranslate2/converters/transformers.py", line 1453, in main
    converter.convert_from_args(args)
  File "/usr/local/lib/python3.10/dist-packages/ctranslate2/converters/converter.py", line 50, in convert_from_args
    return self.convert(
  File "/usr/local/lib/python3.10/dist-packages/ctranslate2/converters/converter.py", line 84, in convert
    raise RuntimeError(
RuntimeError: output directory whisper-large-v2-ct2 already exists, use --force to override


In [10]:
# Run on GPU with FP16
model = WhisperModel(model_path, device="cuda", compute_type="float16")

In [20]:
def create_textfile(video_name):
    results, _  = model.transcribe(video_name, language="ja") #英語ならen 日本語ならja
    with open("transcribe.txt", mode="w") as f:
        for index, _dict in enumerate(results):
            cumsum_time = 0
            start_time = cumsum_time + _dict.start
            end_time = cumsum_time + _dict.end
            text = _dict.text
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int(start_time//(60)), int(end_time//(60))
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            f.write(f'[{s_h:02}:{s_m:02}:{s_s:02} ===> {e_h:02}:{e_m:02}:{e_s:02}] {text}\n')
            print(f'[{s_h:02}:{s_m:02}:{s_s:02} ===> {e_h:02}:{e_m:02}:{e_s:02}] {text}')
    return "transcribe.txt"

css = """
        .gradio-container {
            font-family: 'IBM Plex Sans', sans-serif;
        }
        .container {
            max-width: 730px;
            margin: auto;
            padding-top: 1.5rem;
        }
        #gallery {
            min-height: 22rem;
            margin-bottom: 15px;
            margin-left: auto;
            margin-right: auto;
            border-bottom-right-radius: .5rem !important;
            border-bottom-left-radius: .5rem !important;
        }
        #gallery>div>.h-full {
            min-height: 20rem;
        }

"""

block = gr.Blocks(css=css)

with block:   
    gr.Markdown("whisper Demo")
    with gr.Group():
        with gr.Box():
            with gr.Row(mobile_collapse=False, equal_height=True):
                text_button = gr.Button("映像or音声ファイルをアップしたらここをクリックして文字起こし！").style(
                        margin=False,
                        rounded=(True, True, True, True),
                    )
        video = gr.Audio(source="upload", type="filepath", interactive=True)       
        transcribe = gr.File(label="Transcripts text file")
        text_button.click(create_textfile, inputs=[video], outputs=[transcribe])

block.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://465054432a35d038.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://www.huggingface.co/spaces


[00:00:00 ===> 00:00:03]  A major dam in Ukraine has been badly damaged.
[00:00:03 ===> 00:00:05]  President Zelensky shared this video,
[00:00:05 ===> 00:00:08]  and this dam is part of a hydroelectric power plant
[00:00:08 ===> 00:00:09]  in the town of Novokokovka.
[00:00:09 ===> 00:00:13]  It's in southern Ukraine, in territory controlled by Russia.
[00:00:13 ===> 00:00:15]  And the dam sits on the Dnipro River,
[00:00:15 ===> 00:00:17]  which flows towards the Black Sea.
[00:00:17 ===> 00:00:20]  And this is the scale of what happened.
[00:00:20 ===> 00:00:22]  Compare this satellite image from Sunday
[00:00:22 ===> 00:00:24]  with this image from Tuesday morning.
[00:00:24 ===> 00:00:27]  The dam was overwhelmed with water,
[00:00:27 ===> 00:00:29]  and floods immediately followed.
[00:00:29 ===> 00:00:32]  This is a square in the centre of Novokokovka.
[00:00:32 ===> 00:00:35]  Swans have been swimming past the Palace of Culture.
[00:00:35 ===> 00:00:37]  The BBC has also verifi

(<gradio.routes.App at 0x7fee3c169000>,
 'http://127.0.0.1:7860/',
 'https://465054432a35d038.gradio.app')